In [3]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from urllib.parse import quote
from matplotlib import pyplot as plt
import bs4, ssl
import pandas as pd
import re
import datetime, time # for sleep
import requests
import numpy as np

import xml.etree.ElementTree as et 

import json
import urllib
import urllib.request
import sys

def __Check_Money_Flow_Chart() :

    DateTime = datetime.datetime.today().strftime("%Y%m%d_%H%M")
    File_Name = "Log/Money_Flow.html"
#     File_Name = "Anal_Comp/Money_Flow.html"
    output = open(File_Name, 'a+t')

#     df_Chart = pd.read_csv('Anal_Comp/Money_Flow.txt', sep=", ")
#     df_Chart = pd.read_csv('Anal_Comp/Money_Flow.txt', sep=", ",engine='python',encoding='CP949')
    df_Chart = pd.read_csv('Log/Money_Flow.txt', sep=", ",engine='python',encoding='CP949')
    df_Chart.reset_index()

    #         날짜      지수    등락율      거래량     예탁금   미수금    신용잔고    주식볼륨
    # 0   20200821  2304.59  1.34   865735  523373  3251  157668  730549
    # 1   20200824  2329.83  1.10   649276  517145  3925  157267  731292
    # 2   20200825  2366.73  1.58   771584  517377  2806  156394  735319

    # ls = [type(item) for item in closing_list_A]
    # print(ls) # ---> int
    # ls = [type(item) for item in closing_list_B]
    # print(ls) # ---> <class 'bs4.element.NavigableString'>
    # 천 단위 , 자리수가 있으면 string이고 없으면 int 로 list 생성되는듯..

    df_Chart.rename(columns={'날짜': 'dates'}, inplace = True)
    df_Chart.rename(columns={'거래량': 'Volumes'}, inplace = True)
    df_Chart.rename(columns={'예탁금': 'Balances'}, inplace = True)
    df_Chart.rename(columns={'신용잔고': 'Credits'}, inplace = True)

    closing_list_A = df_Chart["Volumes"].to_list()
    closing_list_B = df_Chart["Balances"].to_list()
    closing_list_C = df_Chart["Credits"].to_list()

    column_list = list(df_Chart.columns)

    date_list = df_Chart["dates"].to_list()
    # for idx, val in enumerate(closing_list_A) :
    #     if type(closing_list_A[idx]) == int or type(closing_list_A[idx]) == float :
    #         continue
    #     else :
    #         val = val.replace(",","")
    #         closing_list_A[idx] = val

    # for idx, val in enumerate(closing_list_B) :
    #     if type(closing_list_B[idx]) == int or type(closing_list_B[idx]) == float :
    #         continue
    #     else :
    #         val = val.replace(",","")
    #         closing_list_B[idx] = val

    print("# after replace")
    print(closing_list_A)
    print(closing_list_B)
    print(closing_list_C)

    # print(date_list.reverse()) # ---> None
    # print(reversed(date_list)) # ---> <list_reverseiterator object at 0x12235bbb0>
    # date_list.reverse() # ---> None
    # print(reversed(date_list)) # ---> <list_reverseiterator object at 0x1190a7d00>
    # print(date_list[::-1]) # ---> OK

    max_value_a = [float(x) for x in closing_list_A]
    max_value_A = max(max_value_a) * 1.5
    max_value_b = [float(x) for x in closing_list_B]
    max_value_B = max(max_value_b) * 1.5 # ---> ValueError: could not convert string to float: '11,550'

    print(column_list) # ['dates', '지수 ', '등락율', 'Volumes', 'Balances', '미수금', 'Credits', '주식볼륨']
    print("# MAX %s, %s" % (max_value_A, max_value_B))
    print("# MAKE HTML Start")

    output.write("<!--\n")
    output.write("    THIS EXAMPLE WAS DOWNLOADED FROM https://echarts.apache.org/examples/en/editor.html?c=mix-line-bar\n")
    output.write("-->\n")
    output.write("<!DOCTYPE html>\n")
    output.write("<html style=\"height: 100%\">\n")
    output.write("    <head>\n")
    output.write("        <meta charset=\"utf-8\">\n")
    output.write("    </head>\n")
    output.write("    <body style=\"height: 100%; margin: 0\">\n")
    output.write("        <div id=\"container\" style=\"height: 100%\"></div>\n")
    output.write("\n")
    output.write("        <script type=\"text/javascript\" src=\"https://cdn.jsdelivr.net/npm/echarts@5/dist/echarts.min.js\"></script>\n")
    output.write("        <!-- Uncomment this line if you want to dataTool extension\n")
    output.write("        <script type=\"text/javascript\" src=\"https://cdn.jsdelivr.net/npm/echarts@5/dist/extension/dataTool.min.js\"></script>\n")
    output.write("        -->\n")
    output.write("        <!-- Uncomment this line if you want to use gl extension\n")
    output.write("        <script type=\"text/javascript\" src=\"https://cdn.jsdelivr.net/npm/echarts-gl@2/dist/echarts-gl.min.js\"></script>\n")
    output.write("        -->\n")
    output.write("        <!-- Uncomment this line if you want to echarts-stat extension\n")
    output.write("        <script type=\"text/javascript\" src=\"https://cdn.jsdelivr.net/npm/echarts-stat@latest/dist/ecStat.min.js\"></script>\n")
    output.write("        -->\n")
    output.write("        <!-- Uncomment this line if you want to use map\n")
    output.write("        <script type=\"text/javascript\" src=\"https://cdn.jsdelivr.net/npm/echarts@5/map/js/china.js\"></script>\n")
    output.write("        <script type=\"text/javascript\" src=\"https://cdn.jsdelivr.net/npm/echarts@5/map/js/world.js\"></script>\n")
    output.write("        -->\n")
    output.write("        <!-- Uncomment these two lines if you want to use bmap extension\n")
    output.write("        <script type=\"text/javascript\" src=\"https://api.map.baidu.com/api?v=2.0&ak=<Your Key Here>\"></script>\n")
    output.write("        <script type=\"text/javascript\" src=\"https://cdn.jsdelivr.net/npm/echarts@5/dist/extension/bmap.min.js\"></script>\n")
    output.write("        -->\n")
    output.write("\n")
    output.write("        <script type=\"text/javascript\">\n")
    output.write("var dom = document.getElementById(\"container\");\n")
    output.write("var myChart = echarts.init(dom);\n")
    output.write("var app = {};\n")
    output.write("\n")
    output.write("var option;\n")
    output.write("\n")
    output.write("    option = {\n")
    output.write("        tooltip: {\n")
    output.write("            trigger: 'axis',\n")
    output.write("            axisPointer: {\n")
    output.write("                type: 'cross',\n")
    output.write("                crossStyle: {\n")
    output.write("                    color: '#999'\n")
    output.write("                }\n")
    output.write("            }\n")
    output.write("        },\n")
    output.write("        toolbox: {\n")
    output.write("            feature: {\n")
    output.write("                dataView: {show: true, readOnly: false},\n")
    output.write("                magicType: {show: true, type: ['line', 'bar']},\n")
    output.write("                restore: {show: true},\n")
    output.write("                saveAsImage: {show: true}\n")
    output.write("            }\n")
    output.write("        },\n")
    output.write("\n")

    output.write("        legend: {\n")
    output.write("            data: ['%s', '%s', '%s'],\n" % (column_list[3], column_list[4], column_list[6]))
    output.write("        },\n")
    output.write("        xAxis: [\n")
    output.write("            {\n")
    output.write("                type: 'category',\n")
    output.write("                data: [")
    for item in date_list :
        output.write("'%s'," % item)
    output.write("                ],\n")
    output.write("                axisPointer: {\n")
    output.write("                    type: 'shadow'\n")
    output.write("                }\n")
    output.write("            }\n")
    output.write("        ],\n")

    output.write("        yAxis: [\n")
    output.write("            {\n")
    output.write("                type: 'value',\n")
    output.write("                name: '%s',\n" % column_list[3])
    output.write("                min: 0,\n")
    output.write("                max: %s,\n" % str(max_value_A))
    output.write("                interval: 50,\n")
    output.write("                axisLabel: {\n")
    output.write("                    formatter: '{value} 원'\n")
    output.write("                }\n")
    output.write("            },\n")
    output.write("            {\n")
    output.write("                type: 'value',\n")
    output.write("                name: '%s',\n" % column_list[4])
    output.write("                min: 0,\n")
    output.write("                max: %s,\n" % str(max_value_B))
    output.write("                interval: 50,\n")
    output.write("                axisLabel: {\n")
    output.write("                    formatter: '{value} 원'\n")
    output.write("                }\n")
    output.write("            }\n")
    output.write("    ],\n")

    output.write("    series: [\n")
    output.write("        {\n")
    output.write("            name: '%s',\n" % column_list[3])
    output.write("            type: 'bar',\n")
    output.write("            data: [\n")
    for x in closing_list_A :
        output.write("'%s'," % x)
    output.write("        ]\n")
    output.write("        },\n")

    output.write("        {\n")
    output.write("            name: '%s',\n" % column_list[4])
    output.write("            type: 'line',\n")
    output.write("            data: [\n")
    for x in closing_list_B  :
        output.write("'%s'," % x)
    output.write("    ]\n")
    output.write("        },\n")

    output.write("        {\n")
    output.write("            name: '%s',\n" % column_list[6])
    output.write("            type: 'line',\n")
    output.write("            yAxisIndex: 1,\n")
    output.write("            data: [\n")
    for x in closing_list_C  :
        output.write("'%s'," % x)
    output.write("    ]\n")
    output.write("        }\n")

    output.write("    ]\n")
    output.write("};\n")

    output.write("if (option && typeof option === 'object') {\n")
    output.write("    myChart.setOption(option);\n")
    output.write("}\n")
    output.write("        </script>\n")
    output.write("    </body>\n")
    output.write("</html>\n")

    print("# MAKE HTML Done")

    output.close()

if(__name__ == '__main__') :

    __Check_Money_Flow_Chart()

# after replace
[865735, 649276, 771584, 771584, 956477, 913917, 1297312, 1647817, 1085245, 1429273, 981546, 772938, 853740, 1041993, 1051028, 735041, 714881, 697961, 605977, 646218, 763618, 861754, 737994, 961726, 724920, 901176, 920205, 931645, 874293, 812837, 890384, 822267, 666941, 705770, 509768, 795585, 678501, 887445, 702037, 1354862, 1464168, 1272038, 1072383, 1239135, 1267494, 923504, 976483, 1129500, 1649443, 1364506, 1410104, 1387435, 1196041, 1311831, 1151129, 1778896, 1499265, 1415239, 1252304, 1227651, 1215239, 1113630, 1015484, 1139513, 1260761, 1121346, 1030890, 1006249, 1026510, 1519911, 1793418, 1524654, 1712525, 1390617, 1578042, 1233182, 1331079, 1353312, 1192695, 1093652, 873023, 924582, 995071, 969262, 887313, 901788, 1460945, 1277207]
[523373, 517145, 517377, 517377, 523168, 538780, 547561, 605270, 595563, 486257, 473964, 553742, 556630, 547780, 556439, 559513, 552769, 556568, 548187, 538801, 580313, 491950, 477330, 542160, 555432, 543438, 540095, 527264, 528083,